In [29]:
#CONDA ENV base_conda (python 3.9.7)
#Import packages
#---------------------------------------
import sys
import os
import glob
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm


#Import your modules
#---------------------------------------
import te_rna_f as ter
sys.path.insert(1, '/cndd3/dburrows/CODE/admin_tools/')
from admin_tools import admin_functions as adm

# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
s_fig = '/cndd3/dburrows/FIGS/'

%load_ext autoreload
sys.version

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) \n[GCC 9.4.0]'

# Make new directories

In [30]:
curr = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM_old/Sample*')
curr = [os.path.basename(i) for i in curr]
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/')
for c in curr:
    os.mkdir(c)

# in_path= "/cndd3/dburrows/DATA/te/rna/PE.bam/"
# out_path = "/cndd/dburrows/DATA/te/rna/PE.counts/TET/"

# samp_l = glob.glob(in_path+'/*Samp*')

# for s in samp_l:
#     #find base directory of s
#     base = os.path.basename(s)
#     if not os.path.exists(out_path+'/'+base):
#         os.mkdir(out_path+'/'+base)
# #5, 8, 9

In [93]:
#symlink files 
curr = glob.glob("/cndd/dburrows/DATA/te/rna/PE.bam_multi20/" + '*Samp*')
nex = glob.glob('/datasets/Public_Datasets/Dracheva_PsychEncode_development/processed/rna_seq/' + '*-*')

for c in curr:
    short = os.path.basename(c).split('_')[-1]
    prac = [i for i in nex if short in i]
    if len(prac) == 1:
         os.symlink(prac[0] + '/Aligned.sortedByCoord.out.bam', c + '/Aligned.sortedByCoord.out.bam')
    elif len(prac) < 1:
        short = short.upper()
        prac = [i for i in nex if short in i]
        if len(prac) == 1:
            os.symlink(prac[0] + '/Aligned.sortedByCoord.out.bam', c + '/Aligned.sortedByCoord.out.bam')
        else:
            if short == 'HCT16HECA028-GABA':
                short = 'HCK16HECA028-GABA'
            if short == 'HCT16HECA028-GLU':
                short = 'HCK16HECA028-GLU'
            prac = [i for i in nex if short in i]
            os.symlink(prac[0] + '/Aligned.sortedByCoord.out.bam', c + '/Aligned.sortedByCoord.out.bam')




In [105]:
#Delete all files in directory
os.chdir('/cndd3/dburrows/DATA/te/rna/CZI.counts/ATEM/pseudobulk')
curr = glob.glob('**')
for c in curr:
    #remove all contents of each directory
    shutil.rmtree(c)


# Check for log files

In [37]:
import glob
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/')
curr_ = glob.glob('*Sa*')
for c in curr_:
    os.chdir(c)
    atem = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + c + '/*ATEM*')
    log = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + c + '/*log*')
    if not atem or not log:
        print(c)
    if log:
        if 'All reads accounted for' not in np.load(log[0]):
            print(c)

    os.chdir('..')

In [28]:
import numpy as np
'All reads accounted for' in np.load(log[0])

True

# Save config files for snakemake

In [27]:
my_dict = {
    "in_path": "/cndd3/dburrows/DATA/te/rna/PE.bam", 
    "out_path": "/cndd/dburrows/DATA/te/rna/PE.counts/ATEM",
    "bed_plus_path": "/cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.plus.full.sort.bed",
    "bed_minus_path": "/cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.minus.full.sort.bed"
}

In [28]:
import os
import glob
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM')
curr_dict = my_dict.copy()
curr_dict['ID'] = glob.glob('*Samp*GABA*')
adm.make_config(curr_dict, '/cndd3/dburrows/CODE/te_rna_ageing/snakemake/workflow_PE-GABA.ATEM/config.json')

Write successful


In [22]:
curr_dict

{'in_path': '/cndd3/dburrows/DATA/te/rna/PE.bam',
 'out_path': '/cndd/dburrows/DATA/te/rna/PE.counts/ATEM',
 'bed_plus_path': '/cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.plus.full.sort.bed',
 'bed_minus_path': '/cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.minus.full.sort.bed',
 'ID': ['Sample_5077_GABA',
  'Sample_1275_GLU',
  'Sample_5086_GABA',
  'Sample_4379_GLU',
  'Sample_5570_GLU',
  'Sample_5918_GABA',
  'Sample_5744_GLU',
  'Sample_4337_GABA',
  'Sample_5184_GABA',
  'Sample_HCT15HBMA007_GABA',
  'Sample_4337_GLU',
  'Sample_198035-1765_GABA',
  'Sample_5977_GLU',
  'Sample_4428_GABA',
  'Sample_5643_GABA',
  'Sample_6285_GLU',
  'Sample_5976_GABA',
  'Sample_3739_GLU',
  'Sample_4425_GLU',
  'Sample_1277_GABA',
  'Sample_1134_GLU',
  'Sample_5570_GABA',
  'Sample_1133_GLU',
  'Sample_6285_GABA',
  'Sample_1823_GABA',
  'Sample_5161_GABA',
  'Sample_198035-1765_GLU',
  'Sample_179134-1401_GABA',
  'Sample_4321_GLU',
  'Sample_5643_GLU',
  'Sample

In [ ]:
"bed_minus_path": "/cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.minus.full.sort.bed", "ID": ["Sample_5086-GLU", "Sample_4369-GLU", "Sample_5309-GLU", "Sample_4411-GLU", "Sample_5871-GLU", "Sample_5936-GLU", "Sample_1241-GLU", "Sample_5387-GLU", "Sample_5976-GLU", "Sample_4725-GLU", "Sample_1539-GLU", "Sample_Hct16HCQA020-GLU", "Sample_5643-GLU", "Sample_4321-GLU", "Sample_1133-GLU", "Sample_1134-GLU", "Sample_4425-GLU", "Sample_3739-GLU", "Sample_6285-GLU", "Sample_5977-GLU", "Sample_4337-GLU", "Sample_HctZZNA014-GLU", "Sample_5744-GLU", "Sample_5570-GLU", "Sample_4379-GLU", "Sample_Hct15HBMA007-GLU", "Sample_1275-GLU", "Sample_Hct17HEIA010-GLU", "Sample_HctYPA008-GLU", "Sample_6007-GLU", "Sample_1535-GLU", "Sample_5326-GLU", "Sample_6032-GLU", "Sample_198034-1435-GLU", "Sample_5184-GLU", "Sample_5293-GLU", "Sample_1823-GLU", "Sample_Hct16HECA028-GLU", "Sample_1105-GLU", "Sample_4414-GLU", "Sample_179134-1401-GLU", "Sample_5401-GLU", "Sample_4545-GLU", "Sample_5787-GLU", "Sample_1648-GLU", "Sample_5446-GLU", "Sample_4428-GLU", "Sample_Hct15HAWA031-GLU", "Sample_Hct15HBNA032-GLU", "Sample_5077-GLU", "Sample_1277-GLU", "Sample_5451-GLU", "Sample_4332-GLU", "Sample_198035-1765-GLU", "Sample_5161-GLU", "Sample_5617-GLU", "Sample_5918-GLU", "Sample_5606-GLU", "Sample_1848-GLU"]}

In [98]:
curr_dict

{'in_path': '/cndd/dburrows/DATA/te/rna/PE.bam_multi20',
 'out_path': '/cndd/dburrows/DATA/te/rna/PE.counts/RNA-mut',
 'ID': ['Sample_5570-GABA',
  'Sample_5086-GLU',
  'Sample_1823-GABA',
  'Sample_6285-GABA',
  'Sample_1277-GABA',
  'Sample_5976-GABA',
  'Sample_5161-GABA',
  'Sample_4411-GABA',
  'Sample_4369-GLU',
  'Sample_5326-GABA',
  'Sample_HctYPA008-GABA',
  'Sample_5309-GLU',
  'Sample_4411-GLU',
  'Sample_5871-GLU',
  'Sample_5077-GABA',
  'Sample_198034-1435-GABA',
  'Sample_5086-GABA',
  'Sample_Hct15HBNA032-GABA',
  'Sample_5936-GLU',
  'Sample_4337-GABA',
  'Sample_1241-GLU',
  'Sample_5184-GABA',
  'Sample_5918-GABA',
  'Sample_5643-GABA',
  'Sample_5387-GLU',
  'Sample_5976-GLU',
  'Sample_4725-GLU',
  'Sample_4428-GABA',
  'Sample_1539-GLU',
  'Sample_4332-GABA',
  'Sample_4379-GABA',
  'Sample_1848-GABA',
  'Sample_179134-1401-GABA',
  'Sample_Hct16HCQA020-GLU',
  'Sample_5643-GLU',
  'Sample_4321-GLU',
  'Sample_5744-GABA',
  'Sample_5401-GABA',
  'Sample_5293-GABA

In [38]:
[(curr[i], p) for i in range(len(curr)) if p in curr[i]]

[('5451-GABA_ATGGCATG-GGTACCTT_HKVTWDSX3', '5451-GABA')]

In [42]:
prac = [i.split('_')[-1] for i in curr_dict['ID']]
os.chdir('/datasets/Public_Datasets/Dracheva_PsychEncode_development/processed/rna_seq/')
curr = glob.glob('*-*')
for p in prac:
    print(p)
    print([(curr[i], p) for i in range(len(curr)) if p in curr[i]])

5570-GABA
[('5570-GABA_TCTCTACT-GAACCGCG_HKVTWDSX3', '5570-GABA')]
5086-GLU
[('5086-GLU_TCTGTTGG-CCATTCGA_HKVTWDSX3', '5086-GLU')]
1823-GABA
[('1823-GABA_GATCTATC-AGCCTCAT_HKVTWDSX3', '1823-GABA')]
6285-GABA
[('6285-GABA_ACTAAGAT-CCGCGGTT_HKVTWDSX3', '6285-GABA')]
1277-GABA
[('1277-GABA_AAGATACT-ATGTAAGT_HKVTWDSX3', '1277-GABA')]
5976-GABA
[('5976-GABA_GCCACAGG-ATGGCATG_HKVTWDSX3', '5976-GABA')]
5161-GABA
[('5161-GABA_AGGTTATA-CGGAACTG_HKVTWDSX3', '5161-GABA')]
4411-GABA
[('4411-GABA_CAATTAAC-ATATCTCG_HKVTWDSX3', '4411-GABA')]
4369-GLU
[('4369-GLU_TAAGGTCA-CTACGACA_HKVTWDSX3', '4369-GLU')]
5326-GABA
[('5326-GABA_ATGTAAGT-CATAGAGT_HKVTWDSX3', '5326-GABA')]
HctYPA008-GABA
[]
5309-GLU
[('5309-GLU_GCGCAAGC-CGGCGTGA_HKVTWDSX3', '5309-GLU')]
4411-GLU
[('4411-GLU_TGGCCGGT-GCGCTCTA_HKVTWDSX3', '4411-GLU')]
5871-GLU
[('5871-GLU_ACGCACCT-CCTTCACC_HKVTWDSX3', '5871-GLU')]
5077-GABA
[('5077-GABA_ACACTAAG-ATATGGAT_HKVTWDSX3', '5077-GABA')]
198034-1435-GABA
[('198034-1435-GABA_TATCGCAC-ACACTAAG_HKVT

In [18]:
curr_dict

{'in_path': '/cndd3/dburrows/DATA/te/rna/PE.bam',
 'out_path': '/cndd/dburrows/DATA/te/rna/PE.counts/RNA-mut',
 'ID': ['Sample_5570-GABA',
  'Sample_5086-GLU',
  'Sample_1823-GABA',
  'Sample_6285-GABA',
  'Sample_1277-GABA',
  'Sample_5976-GABA',
  'Sample_5161-GABA',
  'Sample_4411-GABA',
  'Sample_4369-GLU',
  'Sample_5326-GABA',
  'Sample_HctYPA008-GABA',
  'Sample_5309-GLU',
  'Sample_4411-GLU',
  'Sample_5871-GLU',
  'Sample_5077-GABA',
  'Sample_198034-1435-GABA',
  'Sample_5086-GABA',
  'Sample_Hct15HBNA032-GABA',
  'Sample_5936-GLU',
  'Sample_4337-GABA',
  'Sample_1241-GLU',
  'Sample_5184-GABA',
  'Sample_5918-GABA',
  'Sample_5643-GABA',
  'Sample_5387-GLU',
  'Sample_5976-GLU',
  'Sample_4725-GLU',
  'Sample_4428-GABA',
  'Sample_1539-GLU',
  'Sample_4332-GABA',
  'Sample_4379-GABA',
  'Sample_1848-GABA',
  'Sample_179134-1401-GABA',
  'Sample_Hct16HCQA020-GLU',
  'Sample_5643-GLU',
  'Sample_4321-GLU',
  'Sample_5744-GABA',
  'Sample_5401-GABA',
  'Sample_5293-GABA',
  'S

In [34]:
os.chdir('/cndd3/mcuoco/projects/czi_diversity_rnate/results/merge_bam/')
curr_l = glob.glob('**')
bam_ll = []
for curr in curr_l:
    bam_l = glob.glob(curr+'/*.bam')
    print(len(bam_l))
    bam_ll = np.append(bam_ll, bam_l)

19
19
19
19
18
19
19
19
19
19
19
19
19
19
18
19
18
19
19
19
19


In [8]:
import os
import glob
os.chdir('/cndd3/dburrows/DATA/te/rna/PE.bam')
l = glob.glob('*Samp**')
for n in l:
    os.mkdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + n)

In [6]:
l

['Sample_5570-GABA',
 'Sample_5086-GLU',
 'Sample_1823-GABA',
 'Sample_6285-GABA',
 'Sample_1277-GABA',
 'Sample_5976-GABA',
 'Sample_5161-GABA',
 'Sample_4411-GABA',
 'Sample_4369-GLU',
 'Sample_5326-GABA',
 'Sample_HctYPA008-GABA',
 'Sample_5309-GLU',
 'Sample_4411-GLU',
 'Sample_5871-GLU',
 'Sample_5077-GABA',
 'Sample_198034-1435-GABA',
 'Sample_5086-GABA',
 'Sample_Hct15HBNA032-GABA',
 'Sample_5936-GLU',
 'Sample_4337-GABA',
 'Sample_1241-GLU',
 'Sample_5184-GABA',
 'Sample_5918-GABA',
 'Sample_5643-GABA',
 'Sample_5387-GLU',
 'Sample_5976-GLU',
 'Sample_4725-GLU',
 'Sample_4428-GABA',
 'Sample_1539-GLU',
 'Sample_4332-GABA',
 'Sample_4379-GABA',
 'Sample_1848-GABA',
 'Sample_179134-1401-GABA',
 'Sample_Hct16HCQA020-GLU',
 'Sample_5643-GLU',
 'Sample_4321-GLU',
 'Sample_5744-GABA',
 'Sample_5401-GABA',
 'Sample_5293-GABA',
 'Sample_1133-GLU',
 'Sample_1134-GLU',
 'Sample_4425-GLU',
 'Sample_3739-GLU',
 'Sample_6285-GLU',
 'Sample_6007-GABA',
 'Sample_4425-GABA',
 'Sample_1133-GABA

In [2]:
import pandas as pd
bed_pl = pd.read_csv("/cndd3/dburrows/DATA/te/gtf/bed/rmsk.hg38.filt-5ptrim.plus.sort.bed", sep='\t', header=None)



In [4]:
bed_pl

,0,1,2,3,4,5,6,7
0,chr1,24038,24138,+,MIR,MIR,SINE,211
1,chr1,26741,26841,+,AluSp,Alu,SINE,263
2,chr1,31386,31486,+,AluJo,Alu,SINE,297
3,chr1,32791,32891,+,MIR,MIR,SINE,214
4,chr1,39574,39674,+,AluSx,Alu,SINE,300
...,...,...,...,...,...,...,...,...
548906,chrY,57200662,57200762,+,AluSg7,Alu,SINE,308
548907,chrY,57201438,57201538,+,AluSp,Alu,SINE,305
548908,chrY,57202215,57202315,+,AluSz,Alu,SINE,278
548909,chrY,57202519,57202619,+,AluSc,Alu,SINE,308


In [7]:
bed_pl.columns =['Chromosome', 'Start', 'End', 'Strand', 'gene_id', 'family_id', 'class_id', 'length']
bed_pl

,Chromosome,Start,End,Strand,gene_id,family_id,class_id,length
0,chr1,24038,24138,+,MIR,MIR,SINE,211
1,chr1,26741,26841,+,AluSp,Alu,SINE,263
2,chr1,31386,31486,+,AluJo,Alu,SINE,297
3,chr1,32791,32891,+,MIR,MIR,SINE,214
4,chr1,39574,39674,+,AluSx,Alu,SINE,300
...,...,...,...,...,...,...,...,...
548906,chrY,57200662,57200762,+,AluSg7,Alu,SINE,308
548907,chrY,57201438,57201538,+,AluSp,Alu,SINE,305
548908,chrY,57202215,57202315,+,AluSz,Alu,SINE,278
548909,chrY,57202519,57202619,+,AluSc,Alu,SINE,308


In [8]:
#drop last columns
bed_pl = bed_pl.drop(columns=['length'])
bed_pl

,Chromosome,Start,End,Strand,gene_id,family_id,class_id
0,chr1,24038,24138,+,MIR,MIR,SINE
1,chr1,26741,26841,+,AluSp,Alu,SINE
2,chr1,31386,31486,+,AluJo,Alu,SINE
3,chr1,32791,32891,+,MIR,MIR,SINE
4,chr1,39574,39674,+,AluSx,Alu,SINE
...,...,...,...,...,...,...,...
548906,chrY,57200662,57200762,+,AluSg7,Alu,SINE
548907,chrY,57201438,57201538,+,AluSp,Alu,SINE
548908,chrY,57202215,57202315,+,AluSz,Alu,SINE
548909,chrY,57202519,57202619,+,AluSc,Alu,SINE


In [1]:
#Check that all files are present
import glob
import os

full_l = os.listdir('/cndd2/Public_Datasets/Dracheva_PsychEncode_development/raw_May2022/') 
os.chdir('/cndd3/dburrows/DATA/te/rna/PE.bam/')
check_l = glob.glob('*Samp*')

import numpy as np
not_int = np.setxor1d(check_l, full_l)
assert len(full_l) == len(check_l)
assert len(not_int) == 0

In [ ]:
STAR --genomeDir /cndd2/jchien/iGenome/STAR_gencode_v37 --readFilesIn  R1-merge.fastq.gz R1-merge.fastq.gz --outSAMunmapped None --outFilterType BySJout --outSAMattributes All --outFilterMultimapNmax 100 --outFilterMismatchNmax 999 --outFilterMismatchNoverReadLmax 0.04 --alignIntronMin 20 --alignIntronMax 1000000 --alignMatesGapMax 1000000 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 --sjdbScore 1 --readFilesCommand zcat --runThreadN 16--genomeLoad LoadAndKeep --limitBAMsortRAM 10000000000 --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --winAnchorMultimapNmax 200 --outMultimapperOrder Random --outSAMmultNmax -1

In [76]:
# Check which files went to completion
import glob
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/')
curr_ = glob.glob('*Sa*')
for c in curr_:
    os.chdir(c)
    atem = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + c + '/*ATEM*')
    log = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + c + '/*log*')
    if not atem or not log:
        print(c)
    if log:
        if 'All reads accounted for' not in np.load(log[0]):
            print(c)

    os.chdir('..')